# 13F爬蟲

1. 挑出想要看的機構與其CIK     
範例 : https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=0001350694&owner=exclude&count=40
    
2. 輸入CIK後找出每季13f資料，並抓出其xml(通常是第4個, index = 3)  
    範例 : https://www.sec.gov/Archives/edgar/data/1350694/000117266122001788/infotable.xml
    
3. 整理xml資料，輸進excel

ref : https://github.com/CodeWritingCow/sec-web-scraper-13f/blob/master/scraper.py

In [185]:
import numpy as np
import pandas as pd
import re
import requests
import time
from bs4 import BeautifulSoup

# 前置作業

In [186]:
# import warnings
# warnings.filterwarnings('ignore')
# warnings.warn('DelftStack')
# warnings.warn('Do not show this message')

In [187]:
sec_url = 'https://www.sec.gov'

In [188]:
headers = {
    
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36',
    'Accept-Encoding': 'gzip, deflate, br',
    'HOST': 'www.sec.gov',
    
}

# 公司

## 每個公司的各自季度報告

In [189]:
#更換cik代號可抓取不同公司
#請參考cik.csv

url = "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=0000921669&type=13F-HR&dateb=&owner=exclude&count=40&search_text="
r = requests.get(url, headers=headers)

In [190]:
soup = BeautifulSoup(r.text, "html.parser")

#繳交日期
date = []

for i in range(len(soup.text.split("\n"))):
    if len(soup.text.split("\n")[i]) == 10: 
        if(soup.text.split("\n")[i][0] == "2") & (soup.text.split("\n")[i][4] == "-"):
            date.append(soup.text.split("\n")[i])

#tags為所有報告之相對路徑
tags = soup.findAll('a', id="documentsbutton")

report = []

for i in range(len(tags)):
    report.append(sec_url + tags[i]['href'])

## 各季度報告跟抓每季的第三個xml檔

In [191]:
xmls = []

for i in range(len(report)):
    r2 = requests.get(report[i], headers = headers)
    s2 = BeautifulSoup(r2.text, "html.parser")
    t2 = s2.findAll('a', attrs={'href': re.compile('xml')})
    
    try :
        xml_url = sec_url + t2[3].get('href')
        xmls.append(xml_url)
            
        print(str(date[i]) + " success")
    
    except :
        print(str(date[i]) + " no data")
    
    time.sleep(3)

2022-08-15 success
2022-05-13 success
2022-02-14 success
2021-11-15 success
2021-08-13 success
2021-05-14 success
2021-02-12 success
2020-11-13 success
2020-08-14 success
2020-05-15 success
2020-02-14 success
2019-11-14 success
2019-08-14 success
2019-05-15 success
2019-02-14 success
2018-11-14 success
2018-08-14 success
2018-05-15 success
2018-02-14 success
2017-11-14 success
2017-08-14 success
2017-05-15 success
2017-02-14 success
2016-11-14 success
2016-08-15 success
2016-05-16 success
2016-02-16 success
2015-11-16 success
2015-08-14 success
2015-05-15 success
2015-02-17 success
2014-11-14 success
2014-08-14 success
2014-05-15 success
2014-02-14 success
2013-11-14 success
2013-08-14 success
2013-05-15 no data
2013-02-14 no data
2012-11-15 no data


In [192]:
# s2 = BeautifulSoup(r2.text, "html.parser")
# t2 = s2.findAll('a', attrs={'href': re.compile('xml')})
# xml_url = t2[3].get('href')
# xml_url

## 資料

In [193]:
a = pd.DataFrame(columns = ['date', 'Name of Issuer', "Value (x$1000)"])

for i in range(len(xmls)):
    
#     #臨時更換投資組合需跳過
#     if (i == 4) | (i == 30) :
        
#         print(str(date[i]) + " is a provisional declaration")
        
#         continue
    
    #每筆資料
    r3 = requests.get(xmls[i], headers = headers)
    soup_xml = BeautifulSoup(r3.content, "lxml")

    issuers = soup_xml.body.findAll(re.compile('nameofissuer'))
    values = soup_xml.body.findAll(re.compile('value'))

    data = pd.DataFrame(columns = ['Name of Issuer', "Value (x$1000)"])

    for issuer, value in zip(issuers, values):

        row = {

            "Name of Issuer": issuer.text,
            "Value (x$1000)": float(value.text),

                }

        data = data.append(row, ignore_index = True)

    data = data.groupby("Name of Issuer").sum().reset_index()
    data['porpotion'] = np.round(data['Value (x$1000)'] / data['Value (x$1000)'].sum(), 4) *100
    data = data.sort_values('porpotion', ascending = False).head(20)
    data['date'] = date[i]

    
    a = pd.concat([a, data], axis = 0)
    print(str(date[i]) + " seasonal filing")

    time.sleep(2)

2022-08-15 seasonal filing
2022-05-13 seasonal filing
2022-02-14 seasonal filing
2021-11-15 seasonal filing
2021-08-13 seasonal filing
2021-05-14 seasonal filing
2021-02-12 seasonal filing
2020-11-13 seasonal filing
2020-08-14 seasonal filing
2020-05-15 seasonal filing
2020-02-14 seasonal filing
2019-11-14 seasonal filing
2019-08-14 seasonal filing
2019-05-15 seasonal filing
2019-02-14 seasonal filing
2018-11-14 seasonal filing
2018-08-14 seasonal filing
2018-05-15 seasonal filing
2018-02-14 seasonal filing
2017-11-14 seasonal filing
2017-08-14 seasonal filing
2017-05-15 seasonal filing
2017-02-14 seasonal filing
2016-11-14 seasonal filing
2016-08-15 seasonal filing
2016-05-16 seasonal filing
2016-02-16 seasonal filing
2015-11-16 seasonal filing
2015-08-14 seasonal filing
2015-05-15 seasonal filing
2015-02-17 seasonal filing
2014-11-14 seasonal filing
2014-08-14 seasonal filing
2014-05-15 seasonal filing
2014-02-14 seasonal filing
2013-11-14 seasonal filing
2013-08-14 seasonal filing


In [194]:
a.to_csv("C:\\Users\\internuser1\\Desktop\\13f 爬蟲\\Icahn Carl C.csv", index = False)

In [195]:
# data = data.groupby("Name of Issuer").sum().reset_index()
# data['porpotion'] = np.round(data['Value (x$1000)'] / data['Value (x$1000)'].sum(), 4) *100
# data = data.sort_values('porpotion', ascending = False).head(20)

In [196]:
# r3 = requests.get(a, headers = headers)

In [197]:
# soup_xml = BeautifulSoup(r3.content, "lxml")

In [198]:
# issuers = soup_xml.body.findAll(re.compile('nameofissuer'))
# cusips = soup_xml.body.findAll(re.compile('cusip'))
# values = soup_xml.body.findAll(re.compile('value'))
# sshprnamts = soup_xml.body.findAll('sshprnamt')
# sshprnamttypes = soup_xml.body.findAll(re.compile('sshprnamttype'))
# investmentdiscretions = soup_xml.body.findAll(re.compile('investmentdiscretion'))
# soles = soup_xml.body.findAll(re.compile('sole'))
# shareds = soup_xml.body.findAll(re.compile('shared'))
# nones = soup_xml.body.findAll(re.compile('none'))

In [199]:
# berksha = pd.DataFrame(columns = ['Name of Issuer', "Value (x$1000)"])

# for issuer, value in zip(issuers, values):
        
#     row = { 
#             "Name of Issuer": issuer.text,
#             "Value (x$1000)": float(value.text),
            
#             }
    
#     berksha = berksha.append(row, ignore_index = True)

In [200]:
# berksha = berksha.groupby("Name of Issuer").sum().reset_index()
# berksha['porpotion'] = np.round(berksha['Value (x$1000)'] / berksha['Value (x$1000)'].sum(), 4) *100
# berksha = berksha.sort_values('porpotion', ascending = False).head(20)
# berksha['date'] = "2022-08-14"
# berksha